In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
import joblib
import os

# 檢查裝置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# --- 設定路徑 ---
data_path = '../data/processed/youbike_weather_merged.csv'
info_path = '../data/raw/station_info.csv'

# 1. 讀取主訓練資料
if os.path.exists(data_path):
    df = pd.read_csv(data_path)
    df['record_time'] = pd.to_datetime(df['record_time'])
    print(f"[Info] 成功讀取訓練資料，共 {len(df)} 筆")
else:
    raise FileNotFoundError(f"找不到訓練資料：{data_path}")

# 2. 讀取站點資訊 (用來查中文站名)
if os.path.exists(info_path):
    # 防呆：嘗試用 Tab 分隔讀取，失敗則用逗號
    try:
        df_info = pd.read_csv(info_path, sep='\t')
        if len(df_info.columns) <= 1: df_info = pd.read_csv(info_path)
    except:
        df_info = pd.read_csv(info_path)
        
    # 建立對照表：station_no -> 中文名稱 (去除 YouBike2.0_ 前綴)
    name_map = dict(zip(df_info['station_no'].astype(str), df_info['name_tw'].str.replace('YouBike2.0_', '')))
    print(f"[Info] 成功讀取站點資訊檔")
else:
    print(f"[Warning] 找不到站點資訊檔，將使用預設名稱")
    name_map = {}

Using device: cpu
[Info] 成功讀取訓練資料，共 4377102 筆
[Info] 成功讀取站點資訊檔


In [2]:
# --- 定義降雨分級函數 ---
def get_rain_category(val):
    if val == 0: return 0      # 無雨
    elif val <= 2: return 1    # 毛毛雨 (Drizzle)
    elif val <= 10: return 2   # 小雨/中雨 (Rain)
    else: return 3             # 大雨 (Heavy)

# 確保 rain 欄位無空值
df['rain'] = df['rain'].fillna(0)

# 應用函數產生新特徵
df['Rain_Cat'] = df['rain'].apply(get_rain_category)

print(" 特徵工程完成：已新增 'Rain_Cat' (降雨分級)")
print(df[['record_time', 'rain', 'Rain_Cat']].head())

 特徵工程完成：已新增 'Rain_Cat' (降雨分級)
          record_time  rain  Rain_Cat
0 2025-12-09 21:31:29   0.0         0
1 2025-12-09 21:31:29   0.0         0
2 2025-12-09 21:31:29   0.0         0
3 2025-12-09 21:31:29   0.0         0
4 2025-12-09 21:31:29   0.0         0


In [16]:
# ==========================================
# 3. 執行分區選站邏輯 (完整 13 區)
# ==========================================
selected_stations = []
station_info_map = {} # Dashboard 專用字典

if 'district' in df.columns:
    districts = df['district'].unique()
    print(f"發現行政區: {districts}")
    
    for dist in districts:
        # 找出該行政區的所有資料
        dist_df = df[df['district'] == dist]
        
        if not dist_df.empty:
            # 找出該區出現次數最多者 (流量最大代表站)
            top_station = dist_df['station_no'].value_counts().idxmax()
            
            # 強制轉為字串儲存，避免 ID 型態混亂
            station_id_str = str(top_station)
            selected_stations.append(station_id_str)
            
            # 查真實中文名字
            real_name = name_map.get(station_id_str, f"{dist}代表站")
            
            # 組合顯示名稱： "捷運科技大樓站 (大安區)"
            station_info_map[station_id_str] = f"{real_name} ({dist})"
            
    # 這邊我們不再切前 5 個，而是保留全部 (13個)
    all_target_stations = selected_stations

else:
    print("[Error] 錯誤：找不到 'district' 欄位")
    all_target_stations = []

print(f" 最終鎖定 {len(all_target_stations)} 個分區代表站點進行訓練")
print(f" 這些站點將會出現在 Dashboard 下拉選單中。")

發現行政區: ['大安區' '大同區' '士林區' '文山區' '中正區' '中山區' '內湖區' '北投區' '松山區' '南港區' '信義區' '萬華區'
 '臺大公館校區']
 最終鎖定 13 個分區代表站點進行訓練
 這些站點將會出現在 Dashboard 下拉選單中。


In [17]:
# ==========================================
# 4. 建立 Mapping 與 補值 (針對 13 站)
# ==========================================

# 1. 篩選資料 (只保留這 13 個代表站)
df_filtered = df[df['station_no'].astype(str).isin(all_target_stations)].copy()

# 2. 製作站點 ID Mapping (0, 1, 2, ..., 12)
# 鍵 (Key) 必須是字串，這樣 API 接收前端傳來的字串 ID 才找得到
station_mapping = {str(station): idx for idx, station in enumerate(all_target_stations)}
df_filtered['station_idx'] = df_filtered['station_no'].astype(str).map(station_mapping)

print("✅ Station Mapping Created:", station_mapping)

# 3. 執行補值 (包含新的 Rain_Cat)
features_to_fill = ['bikes_available', 'temperature', 'rain', 'Rain_Cat']

# 先排序，確保時間序列正確
df_filtered = df_filtered.sort_values(['station_idx', 'record_time'])

# 針對每個站點獨立進行補值 (線性插值 + 前後填補)
df_filtered[features_to_fill] = df_filtered.groupby('station_idx')[features_to_fill].transform(
    lambda x: x.interpolate(method='linear').ffill().bfill()
)

print(f"📊 資料過濾完成，總樣本數: {len(df_filtered)}")

✅ Station Mapping Created: {'500101037': 0, '500103021': 1, '500104112': 2, '500105044': 3, '500106074': 4, '500107111': 5, '500108169': 6, '500109016': 7, '500110090': 8, '500111079': 9, '500112087': 10, '500113059': 11, '500119072': 12}
📊 資料過濾完成，總樣本數: 33311


In [18]:
# 設定特徵欄位 (共 4 個)
feature_cols = ['bikes_available', 'temperature', 'rain', 'Rain_Cat']

# 1. 數據縮放 (Scaler)
scaler = MinMaxScaler()
df_filtered[feature_cols] = scaler.fit_transform(df_filtered[feature_cols])

# 2. 製作滑動視窗 (Sliding Window)
def create_multistation_dataset(data, time_steps=3):
    X_list, y_list = [], []
    
    for station_idx in data['station_idx'].unique():
        station_data = data[data['station_idx'] == station_idx]
        
        values = station_data[feature_cols].values # 數值特徵
        ids = station_data['station_idx'].values   # 站點 ID
        
        for i in range(len(values) - time_steps):
            # 取過去 3 小時的特徵
            seq_values = values[i:i+time_steps]
            # 取對應的 ID (重塑形狀以利拼接)
            seq_ids = ids[i:i+time_steps].reshape(-1, 1)
            
            # 合併：[特徵(4) + ID(1)] = 5 個欄位
            combined_input = np.hstack((seq_values, seq_ids))
            
            # Label: 下一時刻的 bikes_available
            target = values[i + time_steps, 0]
            
            X_list.append(combined_input)
            y_list.append(target)
            
    return np.array(X_list), np.array(y_list)

# 執行轉換
TIME_STEPS = 3
X, y = create_multistation_dataset(df_filtered, TIME_STEPS)

# 轉成 PyTorch Tensor 並丟進 GPU/CPU
X_tensor = torch.FloatTensor(X).to(device)
y_tensor = torch.FloatTensor(y).reshape(-1, 1).to(device)

print(f" 序列資料製作完成。Input Shape: {X_tensor.shape}")
# 預期結果: (樣本數, 3, 5) -> 3是時間步, 5是特徵數(4數值+1ID)

 序列資料製作完成。Input Shape: torch.Size([33272, 3, 5])


In [19]:
class MultiStationLSTM(nn.Module):
    def __init__(self, num_stations, input_size=4, hidden_size=64, output_size=1, embedding_dim=5):
        super(MultiStationLSTM, self).__init__()
        
        # 1. 站點嵌入層 (把 0~4 的 ID 轉成向量)
        self.station_embedding = nn.Embedding(num_stations, embedding_dim)
        
        # 2. LSTM 層
        # 輸入維度 = 數值特徵(4) + Embedding(5) = 9
        self.lstm_input_size = input_size + embedding_dim
        self.lstm = nn.LSTM(self.lstm_input_size, hidden_size, batch_first=True)
        
        # 3. 輸出層
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x shape: (batch, time, 5) 
        # 切分資料：前4個是數值，最後1個是ID
        numerical_features = x[:, :, :4] 
        station_ids = x[:, :, 4].long()
        
        # 嵌入運算
        station_embedded = self.station_embedding(station_ids)
        
        # 拼接 (4 + 5 = 9)
        combined_input = torch.cat((numerical_features, station_embedded), dim=2)
        
        # LSTM 運算
        out, _ = self.lstm(combined_input)
        
        # 取最後一個時間點的輸出
        out = out[:, -1, :]
        out = self.dropout(out)
        out = self.fc(out)
        return out

print(" 模型架構定義完成")

 模型架構定義完成


In [20]:
# 初始化模型
num_stations = len(station_mapping)
model = MultiStationLSTM(num_stations=num_stations, input_size=4).to(device)

# 設定超參數
learning_rate = 0.001
num_epochs = 100
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print(" 開始訓練模型...")
model.train()

for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    # Forward
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    
    # Backward
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print(" 訓練結束！")

 開始訓練模型...
Epoch [10/100], Loss: 0.0623
Epoch [20/100], Loss: 0.0450
Epoch [30/100], Loss: 0.0400
Epoch [40/100], Loss: 0.0346
Epoch [50/100], Loss: 0.0293
Epoch [60/100], Loss: 0.0237
Epoch [70/100], Loss: 0.0177
Epoch [80/100], Loss: 0.0123
Epoch [90/100], Loss: 0.0086
Epoch [100/100], Loss: 0.0071
 訓練結束！


In [21]:
# ==========================================
# 8.  存檔區 (模型與對照表同步)
# ==========================================
import joblib

# 確保儲存目錄存在
save_path = '../api/model_files'
if not os.path.exists(save_path):
    os.makedirs(save_path)

# 1. 儲存模型權重 (.pth)
torch.save(model.state_dict(), os.path.join(save_path, 'youbike_lstm_multistation.pth'))

# 2. 儲存數值縮放器 (.pkl)
joblib.dump(scaler, os.path.join(save_path, 'scaler.pkl'))

# 3. 儲存站點索引對照表 (Key 是字串，這點對 API 至關重要)
joblib.dump(station_mapping, os.path.join(save_path, 'station_mapping.pkl'))

# 4. 儲存 Dashboard 專用字典 (只保留有參與訓練的站點)
final_dashboard_map = {sid: name for sid, name in station_info_map.items() if sid in station_mapping}
joblib.dump(final_dashboard_map, os.path.join(save_path, 'station_info_map.pkl'))

print(" 訓練與存檔完成！")
print(f"現在模型已支援以下 {len(station_mapping)} 個站點的預測：")
for sid, name in final_dashboard_map.items():
    print(f" - {sid}: {name}")

 訓練與存檔完成！
現在模型已支援以下 13 個站點的預測：
 - 500101037: 臺大國青大樓宿舍前 (大安區)
 - 500103021: 大稻埕公園 (大同區)
 - 500104112: 劍潭郵局 (士林區)
 - 500105044: 秀明路一段185巷口 (文山區)
 - 500106074: 王貫英先生紀念圖書館 (中正區)
 - 500107111: 長春龍江路口 (中山區)
 - 500108169: 威剛科技總部 (內湖區)
 - 500109016: 捷運復興崗站旅客服務中心 (北投區)
 - 500110090: 健康公園 (松山區)
 - 500111079: 捷運昆陽站(4號出口) (南港區)
 - 500112087: 松仁路305號 (信義區)
 - 500113059: 長順艋舺大道口 (萬華區)
 - 500119072: 臺大原分所北側 (臺大公館校區)
